* SocketServer体系及编程接口
  - Socket套接字 Socket是一种通用的网络编程接口 和网络层次没有一一对应关系
  
  - 进程间通信 **序列化与反序列化**代价比较大
  - 进程内部 效率会较高(但单个线程效率不高) 不用序列化与反序列化
  
* 服务器端编程步骤
  - 创建Socket对象
  - 绑定IP Address 和 Port  bind()
    - IP：路由 解决主机间通信
    - 端口：和进程绑定(确定唯一进程)
  - 开始监听  listen()
  - 获取用于传送数据的Socket对象
    - socket.accept() -> (socket object, address info)  

* Server端开发
  - socket对象 --> bind((IP, Port)) --> listen --> accept --> close
                                                    |--> recv or send --> close  


newsock1和 newsock2 顺序执行 会被阻塞
下面的 accept方法 和recv方法 是阻塞的 主线程经常被阻塞而不能工作 怎么办？
解决：多线程 IO密集型 GIL不明显

In [ ]:
import socket


# 1. 监听socket
server = socket.socket()  # 监听socket 默认TCP IPv4
# 2. 占用 bind (IP, Port)
server.bind(('127.0.0.1', 60000))  # 绑定只能一次
# 3. 监听
server.listen()  # listen完立即返回
print(server)  # 占文件描述符fd

# 测试客户端 Unix: sokit
newsock1, raddr1 = server.accept()  # accept默认阻塞 等队列中的连接
print(newsock1) # 该socket用来通信 知道对端地址
print(raddr1)
# 发送缓存区 会满就会阻塞 一般不会满
data = newsock1.recv(1024)  # TCP 我们知道对端是谁 阻塞，等数据到来
print(type(data), data)
msg = "Your meg = {}".format(data.decode()).encode()  # UTF-8 C/S两端一定要协商字符串编码规则
newsock1.send(msg)  # 有连接协议 不用sendto
print('=' * 30)

newsock2, raddr2 = server.accept()
print(newsock2)
data = newsock2.recv(1024)
newsock2.send(data + b'~@#')

newsock1.close()
newsock2.close()
server.close()


In [ ]:
* 群聊工具实现

In [ ]:
# v1.0

import socket
import logging


FORMAT = "%(asctime)s %(threadName)s %(thread)d %(message)s"
logging.basicConfig(level=20, format=FORMAT, datefmt='%F %T')


# 实现群聊工具 TCP Server
# Server OSError start stop  OOD 面向对象设计
class ChatServer:
    def __init__(self, ip='localhost', port=9999):  # TCP UDP
        self.addr = ip, port
        self.sock = socket.socket()

    def start(self):
        self.sock.bind(self.addr)
        self.sock.listen()

        newsock, raddr = self.sock.accept()  # 从队列中取一个连接来
        logging.info(newsock)
        while True:  # Echo Server
            data = newsock.recv(1024)
            msg = "Your msg = {}".format(data.decode()).encode()
            newsock.send(msg)

    def stop(self):
        self.sock.close()


cs = ChatServer()  # 监听 IP Port
cs.start()

In [ ]:
Main-Thread -> accept-thread -> thread-1 recv send
                             -> thread-2 recv send
                             -> thread-N recv send

In [ ]:
# v2.0

import socket
import logging
import threading
import datetime


FORMAT = "%(asctime)s %(threadName)s %(thread)d %(message)s"
logging.basicConfig(level=20, format=FORMAT, datefmt='%F %T')


# 实现群聊工具 TCP Server
# Server OSError start stop  OOD 面向对象设计
class ChatServer:
    def __init__(self, ip='localhost', port=9999):  # TCP UDP
        self.addr = ip, port
        self.sock = socket.socket()

    def start(self):
        self.sock.bind(self.addr)
        self.sock.listen()
        threading.Thread(target=self.accept, name='Accept').start()  # 启动1个

    def accept(self):
        while True:  # 循环取到来的连接 放到不同的线程中处理
            newsock, raddr = self.sock.accept()  # 从队列中取一个连接来
            logging.info(newsock)
            threading.Thread(target=self.recv, args=(newsock, raddr)).start()  # non-daemon

    # 每有一个客户端 就甩出一个线程 里面有newsock和对应的客户端通信
    def recv(self, sock, raddr):  # 把这段会阻塞的代码 扔到单独的线程中执行
        while True:  # Echo Server
            data = sock.recv(1024)
            msg = "[{:%F %T}] {}:{} {}".format(
                datetime.datetime.now(),
                *raddr,
                data.decode('gbk')
            ).encode('gbk')
            sock.send(msg)

    def stop(self):
        self.sock.close()


def main():
    cs = ChatServer()  # 监听 IP Port
    cs.start()
    while True:
        cmd = input('>>>')
        if cmd == 'quit':
            cs.stop()
            break
        print(threading.active_count(), *threading.enumerate(), sep='\n')


if __name__ == '__main__':
    main()

In [ ]:
# v3.0 把while True -> 使用threading.Event() -- is_set() set()
# Echo Server 回音

import socket
import logging
import threading
import datetime


FORMAT = "%(asctime)s %(threadName)s %(thread)d %(message)s"
logging.basicConfig(level=20, format=FORMAT, datefmt='%F %T')


# 实现群聊工具 TCP Server
# Server OSError start stop  OOD 面向对象设计
class ChatServer:
    def __init__(self, ip='localhost', port=9999):  # TCP UDP
        self.addr = ip, port
        self.sock = socket.socket()
        self.event = threading.Event()

    def start(self):
        self.sock.bind(self.addr)
        self.sock.listen()
        threading.Thread(target=self.accept, name='Accept').start()  # 启动1个

    def accept(self):
        while not self.event.is_set():  # 循环取到来的连接 放到不同的线程中处理
            newsock, raddr = self.sock.accept()  # 从队列中取一个连接来
            logging.info(newsock)
            threading.Thread(target=self.recv, args=(newsock, raddr)).start()  # non-daemon

    # 每有一个客户端 就甩出一个线程 里面有newsock和对应的客户端通信
    def recv(self, sock, raddr):  # 把这段会阻塞的代码 扔到单独的线程中执行
        while not self.event.is_set():  # Echo Server
            data = sock.recv(1024)
            msg = "[{:%F %T}] {}:{} {}".format(
                datetime.datetime.now(),
                *raddr,
                data.decode('gbk')
            ).encode('gbk')
            sock.send(msg)

    def stop(self):
        self.event.set()  # 结束的时候 event 置为True
        self.sock.close()


def main():
    cs = ChatServer()  # 监听 IP Port
    cs.start()
    while True:
        cmd = input('>>>')
        if cmd == 'quit':
            cs.stop()
            break
        print(threading.active_count(), *threading.enumerate(), sep='\n')


if __name__ == '__main__':
    main()


In [ ]:
# v4.0 
# 实现群发
# todo: 关闭时候的异常捕获 aeecpt线程 和 recv工作线程


import socket
import logging
import threading
import datetime


FORMAT = "%(asctime)s %(threadName)s %(thread)d %(message)s"
logging.basicConfig(level=20, format=FORMAT, datefmt='%F %T')


# 实现群聊工具 TCP Server
# Server OSError start stop  OOD 面向对象设计
class ChatServer:
    def __init__(self, ip='localhost', port=9999):  # TCP UDP
        self.addr = ip, port
        self.sock = socket.socket()
        self.event = threading.Event()
        self.clients = []

    def start(self):
        self.sock.bind(self.addr)
        self.sock.listen()
        threading.Thread(target=self.accept, name='Accept').start()  # 启动1个

    def accept(self):
        while not self.event.is_set():  # 循环取到来的连接 放到不同的线程中处理
            newsock, raddr = self.sock.accept()  # 从队列中取一个连接来
            self.clients.append(newsock)
            logging.info(newsock)
            threading.Thread(target=self.recv, args=(newsock, raddr)).start()  # non-daemon

    # 每有一个客户端 就甩出一个线程 里面有newsock和对应的客户端通信
    def recv(self, sock, raddr):  # 把这段会阻塞的代码 扔到单独的线程中执行
        while not self.event.is_set():  # Echo Server
            data = sock.recv(1024)
            msg = "[{:%F %T}] {}:{} {}".format(
                datetime.datetime.now(),
                *raddr,
                data.decode('gbk')
            ).encode('gbk')
            for c in self.clients:
                c.send(msg)

    def stop(self):
        self.event.set()  # 结束的时候 event 置为True
        for c in self.clients:
            c.close()
        self.sock.close()


def main():
    cs = ChatServer()  # 监听 IP Port
    cs.start()
    while True:
        cmd = input('>>>')
        if cmd == 'quit':
            cs.stop()
            break
        print(threading.active_count(), *threading.enumerate(), sep='\n')
        print(cs.clients)


if __name__ == '__main__':
    main()

**遍历过程中 改变hash表Size [set dict]都有这个问题**

* 遍历过程中 改变hash表Size 会抛出RuntimeError
RuntimeError: dictionary changed size during iteration

In [ ]:
# v5.0 
# 注意几处需要加锁的地方 保证线程安全

import socket
import logging
import threading
import datetime


FORMAT = "%(asctime)s %(threadName)s %(thread)d %(message)s"
logging.basicConfig(level=20, format=FORMAT, datefmt='%F %T')


# 实现群聊工具 TCP Server
# Server OSError start stop  OOD 面向对象设计
class ChatServer:
    def __init__(self, ip='localhost', port=9999):  # TCP UDP
        self.addr = ip, port
        self.sock = socket.socket()
        self.event = threading.Event()
        self.lock = threading.Lock()
        self.clients = {}
        # [] 当sock对象数量增大 使用列表效率低
        # set() 使用集合 不方便观察

    def start(self):
        self.sock.bind(self.addr)
        self.sock.listen()
        threading.Thread(target=self.accept, name='Accept').start()  # 启动1个

    def accept(self):
        while not self.event.is_set():  # 循环取到来的连接 放到不同的线程中处理
            newsock, raddr = self.sock.accept()  # 从队列中取一个连接来
            with self.lock:
                self.clients[raddr] = newsock
            logging.info(newsock)
            threading.Thread(target=self.recv, args=(newsock, raddr)).start()  # non-daemon

    # 每有一个客户端 就甩出一个线程 里面有newsock和对应的客户端通信
    def recv(self, sock, raddr):  # 把这段会阻塞的代码 扔到单独的线程中执行
        while not self.event.is_set():  # Echo Server
            data = sock.recv(1024)  # 抛异常
            print(data, '+++++')  # 客户端主动断开 会发送一个空bytes
            if not data or data == b'quit':
                logging.info('{} bye bye'.format(raddr))
                sock.close()
                # 清理工作
                with self.lock:
                    self.clients.pop(raddr)
                break  # 跳出当前循环 recv终止
            msg = "[{:%F %T}] {}:{} {}".format(
                datetime.datetime.now(),
                *raddr,
                data.decode('gbk')
            ).encode('gbk')
            # 一个线程遍历之后 CPU时间片用完 切到另一个线程需要退出(line:43) 或者增加(line:30)
            # 操作 self.clients 导致线程不安全
            with self.lock:  # 线程安全 必须要等 
                for c in self.clients.values():
                    c.send(msg)

    def stop(self):
        self.event.set()  # 结束的时候 event 置为True
        with self.lock:
            for c in self.clients.values():
                c.close()
        self.sock.close()


def main():
    cs = ChatServer()  # 监听 IP Port
    cs.start()
    while True:
        cmd = input('>>>')
        if cmd == 'quit':
            cs.stop()
            break
        print(threading.active_count(), *threading.enumerate(), sep='\n')
        print(cs.clients.keys())


if __name__ == '__main__':
    main()

In [ ]:
* 同步 --> 优化: 同步变异步 --> 使用队列：kafka、Queue

In [ ]:
# v6.0 
# 处理 客户端异常退出(崩溃) 产生的异常
# 清理self.clients 保存的无用的newsock对象

import socket
import logging
import threading
import datetime


FORMAT = "%(asctime)s %(threadName)s %(thread)d %(message)s"
logging.basicConfig(level=20, format=FORMAT, datefmt='%F %T')


# 实现群聊工具 TCP Server
# Server OSError start stop  OOD 面向对象设计
class ChatServer:
    def __init__(self, ip='localhost', port=9999):  # TCP UDP
        self.addr = ip, port
        self.sock = socket.socket()
        self.event = threading.Event()
        self.lock = threading.Lock()
        self.clients = {}
        # [] 当sock对象数量增大 使用列表效率低
        # set() 使用集合 不方便观察

    def start(self):
        self.sock.bind(self.addr)
        self.sock.listen()
        threading.Thread(target=self.accept, name='Accept').start()  # 启动1个

    def accept(self):
        while not self.event.is_set():  # 循环取到来的连接 放到不同的线程中处理
            newsock, raddr = self.sock.accept()  # 从队列中取一个连接来
            with self.lock:
                self.clients[raddr] = newsock
            logging.info(newsock)
            threading.Thread(target=self.recv, args=(newsock, raddr)).start()  # non-daemon

    # 每有一个客户端 就甩出一个线程 里面有newsock和对应的客户端通信
    def recv(self, sock, raddr):  # 把这段会阻塞的代码 扔到单独的线程中执行
        while not self.event.is_set():  # Echo Server
            try:
                data = sock.recv(1024)  # 抛异常
            except Exception as e:  # SOError
                logging.error(e)  # 连接不可用
                data = b''
            print(data, '+++++')  # 客户端主动断开 会发送一个空bytes
            if not data or data == b'quit':
                logging.info('{} bye bye'.format(raddr))
                sock.close()
                # 清理工作
                with self.lock:
                    self.clients.pop(raddr)
                break  # 跳出当前循环 recv终止
            msg = "[{:%F %T}] {}:{} {}".format(
                datetime.datetime.now(),
                *raddr,
                data.decode('gbk')
            ).encode('gbk')
            # 一个线程遍历之后 CPU时间片用完 切到另一个线程需要退出(line:43) 或者增加(line:30)
            # 操作 self.clients 导致线程不安全
            with self.lock:
                for c in self.clients.values():
                    c.send(msg)

    def stop(self):
        self.event.set()  # 结束的时候 event 置为True
        with self.lock:
            for c in self.clients.values():
                c.close()
        self.sock.close()


def main():
    cs = ChatServer()  # 监听 IP Port
    cs.start()
    while True:
        cmd = input('>>>')
        if cmd == 'quit':
            cs.stop()
            break
        print(threading.active_count(), *threading.enumerate(), sep='\n')
        print(cs.clients.keys())


if __name__ == '__main__':
    main()



In [ ]:
其他线程 recv阻塞的时候 直接关闭 会报错
解决： 1. 异常捕获
      2. 直接使用daemon线程 socket.close()之后 其他recv daemon线程不用管 
         随着一起关闭(把recv和aeecpt线程设置为daemon线程)

* MakeFile
对比socket 
  - 将recv方法 对映成 文件读操作(read|readline)
  - 将send方法 对应成 文件写操作(write)

In [ ]:

import socket


server = socket.socket()
server.bind(('127.0.0.1', 9999))
server.listen()

newsock, client = server.accept()
f = newsock.makefile(mode='rw', encoding='utf-8')  # mode和文件操作稍微不一样 注意区分
print(newsock)
print(f)
data = f.readline()  # str
print(type(data), data)
f.write(data + '###')
f.flush()


# data = newsock.recv(1024)  # bytes
# newsock.send(data)

newsock.close()
server.close()

In [ ]:
* 客户端实现
ChatClient

In [ ]:
# 群聊 客户端版本
import logging
import socket
import threading


# FORMAT = '%(asctime)s %(threadNmae)s %(thread)d %(message)s'
FORMAT = '%(asctime)s %(message)s'
logging.basicConfig(level=20, format=FORMAT, datefmt='%F %T')


class ChatClient:
    def __init__(self, rip='localhost', rport=9999):
        self.__raddr = rip, rport
        self.sock = socket.socket()
        self.event = threading.Event()
        # self.encoding = 'utf-8'  # 与服务端约定编码

    def start(self):
        # 捕获异常 断线重连；连接失败重连
        self.sock.connect(self.__raddr)

        # 模拟client向服务器汇报
        self.send('say hello')
        threading.Thread(target=self.recv, name='recv').start()

    def recv(self):
        while not self.event.is_set():
            data = self.sock.recv(1024)
            logging.info(data)

    def send(self, msg:str):
        self.sock.send(msg.encode())

    def stop(self):
        self.sock.close()


def main():
    cc = ChatClient()
    cc.start()
    while True:
        cmd = input('>>>').strip()
        if cmd == 'quit':
            cc.stop()
            break
        cc.send(cmd)
        print(threading.enumerate())


if __name__ == '__main__':
    main()

* SocketServer

上面socket写的Server 出了消息处理部分 其余大部分都是固定套路

socket编程过于底层 编程虽然有套路 但是想要写出**健壮**的代码还是比较困难的 所以很多语言都对socket底层API进行封装
Python的封装就是sockerserver模块 它是网络服务编程框架 便于企业级快速开发

* 类的继承关系

BaseServer  （基类 约定应该实现的方法）
    |
TCPServer  ->  UnixStreamServer
    |
DDPServer  ->  UnixDatagramServer

SockerServer简化了网络服务器的编写
它有4个同步类：
    - TCPServer
    - UDPServer
    - UnixStreamServer
    - UnixDatagramServer
2个Mixin类：ForkingMixIn(多进程支持) 和 ThreadingMixIn(多线程支持)类 用来支持异步
由此得到：
    - class ForkingUDPServer(ForkingMixIn, UDPServer): pass
    - class ForkingTCPServer(ForkingMixIn, TCPServer): pass
    - class ThreadingUDPServer(ThreadingMixIn, UDPServer): pass
    - class ThreadingTCPServer(ThreadingMixIn, TCPServer): pass

fork是创建多进程 thread是创建多线程
fork需要操作系统支持 Windows不支持

In [ ]:
* socketserver

In [ ]:
import socketserver
import logging
import threading


FORMAT = "%(asctime)s %(threadName)s %(thread)d %(message)s"
logging.basicConfig(level=20, format=FORMAT, datefmt='%F %T')


# 实现一个EchoServer

# 作业：使用socketserver重写群聊软件

# 给你个基类 定义了处理流程
class MyHandler(socketserver.BaseRequestHandler):
    def handle(self) -> None:  # recv方法
        print(self.request)  # socket对象
        print(self.client_address)  # 客户端地址：Port
        print(self.server)  # TCPServer
        print(id(self.server))
        print('=' * 30)

        while True:  # 单线程 永远阻塞在处理第一个连接
            # for i in range(3):  # 单线程 阻塞在handle 三次握手完成 把下一个连接放在队列中等待
            data = self.request.recv(1024)
            msg = "### {} ###".format(data.decode()).encode()
            self.request.send(msg)
            print('-' * 30)
            logging.info(threading.enumerate())


# class BaseServer:
#     def __init__(self, server_address, RequestHandlerClass):
#         """Constructor.  May be extended, do not override."""
#         self.server_address = server_address
#         self.RequestHandlerClass = RequestHandlerClass
#         self.__is_shut_down = threading.Event()
#         self.__shutdown_request = False
#
#     def finish_request(self, request, client_address):
#         """Finish one request by instantiating RequestHandlerClass."""
#         self.RequestHandlerClass(request, client_address, self)  # 可能有若干实例

# 请求是什么？ 客户端发来的数据
# 一个请求时什么？

# socketserver socket编程框架
# server = socketserver.TCPServer(('127.0.0.1', 9999), MyHandler)
server = socketserver.ThreadingTCPServer(('127.0.0.1', 9999), MyHandler)
print(1, id(server))
# server.handle_request()  # 处理客户端请求 一次连接(一次请求)处理完
server.serve_forever()  # IO多路复用
server.server_close()  # close & clean-up

In [ ]:
* 实现一个EchoServer
* 使用socketserver重写群聊软件

In [ ]:
import socketserver
import logging
import threading


FORMAT = "%(asctime)s %(threadName)s %(thread)d %(message)s"
logging.basicConfig(level=20, format=FORMAT, datefmt='%F %T')


# 实现一个EchoServer

# 作业：使用socketserver重写群聊软件

# 给你个基类 定义了处理流程
class EchoHandler(socketserver.BaseRequestHandler):
    def setup(self) -> None:
        super().setup()
        self.event = threading.Event()

    def finish(self) -> None:
        super().finish()
        self.event.set()

    def handle(self) -> None:  # recv方法
        super().handle()
        print('=' * 30)
        while True:  # 单线程 永远阻塞在处理第一个连接
            data = self.request.recv(1024)
            if not data or data == b'quit':
                break
            msg = "### {} ###".format(data.decode()).encode()
            self.request.send(msg)
            print('-' * 30)
            logging.info(threading.enumerate())


# 请求是什么？ 客户端发来的数据
# 一个请求时什么？

# socketserver socket编程框架
# server = socketserver.TCPServer(('127.0.0.1', 9999), MyHandler)
server = socketserver.ThreadingTCPServer(('127.0.0.1', 9999), EchoHandler)
# server.serve_forever()  # IO多路复用
threading.Thread(target=server.serve_forever, name='serve').start()

while True:
    cmd = input('>>>')
    if cmd == 'quit':
        server.server_close()
        break
    print(threading.enumerate())
# server.server_close()  # close & clean-up
print('=' * 30)

In [ ]:
* 使用sockerserver重写群聊软件

In [ ]:
import socketserver
import logging
import threading


FORMAT = "%(asctime)s %(threadName)s %(thread)d %(message)s"
logging.basicConfig(level=20, format=FORMAT, datefmt='%F %T')


class EchoHandler(socketserver.BaseRequestHandler):
    clients = {}

    def setup(self) -> None:
        # setup 资源申请 变量配置
        super().setup()
        self.event = threading.Event()
        self.lock = threading.Lock()
        self.clients[self.client_address] = self.request

    def finish(self) -> None:
        # finally中执行 不管是否异常 都会执行 主要做清理工作
        super().finish()
        self.event.set()
        with self.lock:
            self.clients.pop(self.client_address)

    def handle(self) -> None:
        while True:
            data = self.request.recv(1024)
            if not data or data == b'quit':
                break
            msg = "### {} ###".format(data.decode()).encode()
            with self.lock:
                for sock in self.clients.values():
                    sock.send(msg)


server = socketserver.ThreadingTCPServer(('127.0.0.1', 9999), EchoHandler)
server.daemon_threads = True  # 设置每个handle线程为daemon线程 否则OSError
threading.Thread(target=server.serve_forever, name='serve_forever').start()
while True:
    cmd = input('>>>')
    if cmd == 'quit':
        server.server_close()
        server.shutdown()   # Stops the serve_forever loop. 否则直接退出会有OSError
        break
    print(threading.enumerate())

print("=====END=====")

- **同步** **异步**
  - 函数或方法被调用的时候 调用者是否得到**最终结果**
      - 直接得到**最终结果**的 就是同步调用
      - 不直接得到**最终结果** 就是异步调用

- **阻塞** **非阻塞** 跟时间相关 没有具体的时间标准
  - 函数或方法调用的时候 是否**立即**返回
      - 立即返回就是非阻塞调用 
      - 不立即返回就是阻塞调用

- **区别**
  - 同步 异步与阻塞、非阻塞不相关
  - 同步、异步强调的是 是否得到(最终的)**结果**
  - 阻塞、非阻塞强调的是时间 是否**等待**
  
  
* 异步 中比较常见的三种方式
  - 1. 轮询  给号 调用者轮询，每隔一段时间询问是否ok(调用者比较空闲采用这个)
  - 2. 叫号  调用者去干其他活，被调用者通知调用者 通知机制
  - 3. 回调  callback(js用得非常多) 给号会问：到了您这个号怎么办 给我个方案 e.g add() -> 到我了帮我把add函数执行下
      - 回调 调用者在调用过被调用者的时候，留给被调用者的方案 
      - 调用者走了 说明往往时异步的
      - 经过了一段时间 当最终结果有了 被调用者执行回调函数add
      - 回调不是调用者调用 调用者留下方案，被调用者执行
      

* 同步阻塞
  - 一直等最终结果 饭没好 石化等待 一直等待饭好
    
* 同步非阻塞
  - 不给任何结果 饭没好 不石化(返回一个异常 不算结果)，任何其他结果都不接受(程序没法实现， 返回一个异常则重复调用) 几乎没有利用价值
  
* 异步阻塞
  - 取了号(中间结果) 石化等待 一直等待饭好  几乎没有地方应用，没有效率
 
* 异步非阻塞
  - 取了号(中间结果 未来拿最终结果的凭据 最终结果:可以拿可以不拿) 不石化
  - 怎么回来拿结果 参考上面的三种方式（轮询 叫号 回调）

In [12]:
from concurrent.futures import ThreadPoolExecutor


def calc(ret):
    sum = 0
    for i in range(10000000):
        sum += 1
    print(sum)
    return ret

fs = []
executor = ThreadPoolExecutor(max_workers=4)
with executor:
    for i in range(4):
        # future 拿到了最终的结果吗？ 不是，拿到了未来的"凭据" - 异步
        future = executor.submit(calc, i+100)
        fs.append(future)
for i in fs:
    print(i, type(i), i.result())

10000000
10000000
10000000
10000000
<Future at 0x1a788315948 state=finished returned int> <class 'concurrent.futures._base.Future'> 100
<Future at 0x1a7878836c8 state=finished returned int> <class 'concurrent.futures._base.Future'> 101
<Future at 0x1a787883148 state=finished returned int> <class 'concurrent.futures._base.Future'> 102
<Future at 0x1a7873fb708 state=finished returned int> <class 'concurrent.futures._base.Future'> 103


* 同步IO 异步IO IO多路复用

* IO两个阶段：
IO过程分两阶段：
  1. 数据准备阶段 从设备读取数据到内核空间的缓冲区
  2. 内核空间复制回用户空间进程缓冲区阶段
  
  系统调用： read函数、recv函数
  
* IO模型
  - 同步IO
    同步IO模型包括 
    - 阻塞IO
    
    - 非阻塞IO
    返回异常 不直接返回结果 调用者不石化 而是一直询问"好了没"
      
    - IO多路复用
    也称Event-driven IO 事件驱动模型
    所谓IO多路复用 就是同时监控多个IO 有一个准备好了 就不需要等了开始处理，提高了同时处理IO的能力
    一个IO对象 就理解为 一路，以select为例 将关注的IO操作告诉select函数并调用，进程阻塞，内核"监视"select关注的文件描述符fd，被关注的任何一个fd对象的IO准备好了数据，select返回。再使用read将数据复制到用户进程
      
      - select 是菜准备好了 大师傅喊你 你要自己遍历，一般请况下 selcet最多能监听1024个fd(可以修改)，但是由于select是采用的轮询的方式 当管理IO多了 每次都要遍历全部fd 效率低下 （可以理解为元组 列表）
      - poll 链表 略微增强select
      - epoll是有菜准备好了 大师傅喊你去几号窗口直接打菜 不用自己找（回调）没有管理的fd上限 且是回调机制 不需要遍历 效率很高（共用内存空间 减少复制 回调）
       
    - 信号驱动IO(较少用)
    
  - 异步IO
    系统调用aio

* Python中IO多路复用

  - IO多路复用
    - 大多数操作系统都支持select和poll poll是对select的升级
    - Linux系统内核2.5+ 支持epoll
    - BSD MAC支持kqueue
    - Solaris实现了/dev/poll
    - Windows的IOCP
    
  Python的select库实现了select、poll系统调用 这个基于基本上操作系统都支持 对Linux内核2.5+支持了epoll
  
  开发中选择：
  1. 完全跨平台 使用select poll 但是性能较差
  2. 针对不同操作系统自行选择支持的技术 可以提高IO处理的性能
  
  select维护一个文件描述符数据结构 单个进程使用有上限 通常是1024 线性扫描这个数据结果，效率低
  
  pool和select的区别是内部数据使用链表 没有这个最大限制 但是依然是线性遍历才知道哪个设备就绪了
  
  epoll使用事件通知机制 使用回调机制提高效率
  
  select/poll还要从内核空间复制消息到用户空间 而epoll通过内核空间和用户空间共享一块内存来减少复制
  
  
* 库
  - select    底层模块
  - selectors 高级封装 推荐使用
    - select
    - poll
    - epoll
    - devpoll
    
    selectors.DefaultSelector  # 获得符合本系统最好的IO多路复用
  

In [ ]:
import selectors
import socket


server = socket.socket()  # IO对象
server.bind(('127.0.0.1', 60000))
server.listen()
# 官方需要使用非阻塞IO
server.setblocking(False)
print(id(server))


# 获得符合本系统最好的IO多路复用 fd(网络IO 文件IO) fileno()
# 一个监控者 我们有多路IO 交给他监视
selector = selectors.DefaultSelector()
# <class 'abc.ABCMeta'>
# <class 'selectors.SelectSelector'>


# 注册监视的IO
key = selector.register(server, selectors.EVENT_READ)
# 监视A路IO IO内核中就绪了 你可以读了 你是谁？server这个被监控对象赶紧来读取 执行第二阶段
# 对于listen的socket对象来说 读event就绪什么意思？连接就绪了，队列里面有了
print(key)
# 实例 fileobj：文件对象 ；fd=504 fileibj的fd；
# events=1； data=None
# EVENT_READ = (1 << 0)  0b10
# EVENT_WRITE = (1 << 1) 0b11
print('-' * 30)

events = selector.select()  # 默认永久阻塞 阻塞到它关注的某一路或某几路第一阶段就绪
print(events)  #好的那几路的list item(key, mask)  events & mask 判断是读就绪 还是写就绪
# server.accept()   # x.fileobj.accept() newsock, raddr
for key, mask in events:
    newsock, raddr = key.fileobj.accept()
    print('=' * 30)
    newsock.send(b'welcome')


server.close()


In [ ]:
import selectors
import socket


server = socket.socket()  # IO对象
server.bind(('127.0.0.1', 60000))
server.listen()
# 官方需要使用非阻塞IO
server.setblocking(False)
# 获得符合本系统最好的IO多路复用 fd(网络IO 文件IO) fileno()
# 一个监控者 我们有多路IO 交给他监视
selector = selectors.DefaultSelector()
# <class 'abc.ABCMeta'>
# <class 'selectors.SelectSelector'>

# 注册监视的IO
key = selector.register(server, selectors.EVENT_READ)
# 监视A路IO IO内核中就绪了 你可以读了 你是谁？server这个被监控对象赶紧来读取 执行第二阶段
# 对于listen的socket对象来说 读event就绪什么意思？连接就绪了，队列里面有了
print(key)
# 实例 fileobj：文件对象 ；fd=504 fileibj的fd；
# events=1； data=None
# EVENT_READ = (1 << 0)  0b10
# EVENT_WRITE = (1 << 1) 0b11
print('-' * 30)


while True:
    events = selector.select()  # 默认永久阻塞 阻塞到它关注的某一路或某几路第一阶段就绪
    print(events)  #好的那几路的list item(key, mask)  events & mask 判断是读就绪 还是写就绪
    # server.accept()   # x.fileobj.accept() newsock, raddr
    for key, mask in events:
        if key.fileobj is server:
            print('-' * 30)
            newsock, raddr = key.fileobj.accept()
            selector.register(newsock, selectors.EVENT_READ)
            newsock.send(b'welcome')
        else:
            print('=' * 30)
            data = key.fileobj.recv(1024)  # 各个newsock和客户端地址
            msg = "from {}. msg = {}".format(key.fileobj.getpeername(), data.decode()).encode()
            key.fileobj.send(msg)
        print('+' * 30)

server.close()

In [ ]:
import selectors
import socket


server = socket.socket()  # IO对象
server.bind(('127.0.0.1', 60000))
server.listen()
# 官方需要使用非阻塞IO
server.setblocking(False)
# 获得符合本系统最好的IO多路复用 fd(网络IO 文件IO) fileno()
# 一个监控者 我们有多路IO 交给他监视
selector = selectors.DefaultSelector()
# <class 'abc.ABCMeta'>
# <class 'selectors.SelectSelector'>


def accept(s, mask):
    print('-' * 30)
    newsock, raddr = s.accept()
    selector.register(newsock, selectors.EVENT_READ, recv)
    newsock.send(b'welcome')


def recv(newsock):
    print('=' * 30)
    data = newsock.recv(1024)  # 各个newsock和客户端地址
    msg = "from {}. msg = {}".format(newsock .getpeername(), data.decode()).encode()
    newsock.send(msg)


# 注册监视的IO
key = selector.register(server, selectors.EVENT_READ, accept)
# 监视A路IO IO内核中就绪了 你可以读了 你是谁？server这个被监控对象赶紧来读取 执行第二阶段
# 对于listen的socket对象来说 读event就绪什么意思？连接就绪了，队列里面有了
print(key)
# 实例 fileobj：文件对象 ；fd=504 fileibj的fd；
# events=1； data=None
# EVENT_READ = (1 << 0)  0b10
# EVENT_WRITE = (1 << 1) 0b11
print('-' * 30)

while True:
    events = selector.select()  # 默认永久阻塞 阻塞到它关注的某一路或某几路第一阶段就绪
    print(events)  #好的那几路的list item(key, mask)  events & mask 判断是读就绪 还是写就绪
    # server.accept()   # x.fileobj.accept() newsock, raddr
    for key, mask in events:
        print('+' * 30)
        key.data(key.fileobj, mask)  # 注册的时候 data传入处理函数 简化代码 不用判断
        # if key.data is accept:
        #     key.data(key.fileobj, mask)  # accept(server, 1)
        # else:
        #     key.data(key.fileobj, mask)  # recv(newsock*, 1)

server.close()

In [ ]:
import selectors
import socket


server = socket.socket()  # IO对象
server.bind(('127.0.0.1', 60000))
server.listen()
# 官方需要使用非阻塞IO
server.setblocking(False)
selector = selectors.DefaultSelector()


def accept(s, mask):
    print('-' * 30)
    newsock, raddr = s.accept()
    selector.register(newsock, selectors.EVENT_READ, recv)
    newsock.send(b'welcome')


def recv(newsock):
    print('=' * 30)
    data = newsock.recv(1024)  # 各个newsock和客户端地址
    msg = "from {}. msg = {}".format(newsock.getpeername(), data.decode()).encode()
    newsock.send(msg)
    # httpServer HTTP纯文本协议
    # recv读取请求数据 open打开一个html文件 读取内容直接返回，header自己做一个
    # Django不是Server 是APP
    # Django写的就是recv函数


# 注册监视的IO
key = selector.register(server, selectors.EVENT_READ, accept)

while True:
    events = selector.select()  # 默认永久阻塞 阻塞到它关注的某一路或某几路第一阶段就绪
    print(events)  # 好的那几路的list item(key, mask)  events & mask 判断是读就绪 还是写就绪
    for key, mask in events:
        # 如果self.recv is key.data --> False(跟描述器有关), 要使用self.recv == key.data
        print(key.data, key.fileobj is server)
        key.data(key.fileobj, mask)

# 我们现在有3路监控的IO对象 请问有几个线程搞定这个事情的？ 1个线程
# socketserver内部的ThreadingTCPServer 就是用的IO多路复用
# 大大的减少了线程的使用 减少了线程切换的开销(线程越多 开销越大)

In [ ]:
# IO多路复用实现群聊 还有bug待修复

import selectors
import socket


class ChatServer:
    def __init__(self, ip='127.0.0.1', port=9999):
        self._addr = ip, port
        self.socket = socket.socket()
        self.socket.setblocking(False)
        self.selector = selectors.DefaultSelector()
        self.clients = {}

    def start(self):
        self.socket.bind(self._addr)
        self.socket.listen()
        self.selector.register(self.socket, selectors.EVENT_READ, data=self.accept)

    def accept(self, sock, mask):
        newsock, raddr = sock.accept()
        self.clients[raddr] = newsock
        print(self.clients.keys())
        self.selector.register(newsock, selectors.EVENT_READ, data=self.recv)

    def recv(self, newsock, mask):
        data = newsock.recv(1024)
        if data == b'quit' or not data:
            self.clients.pop(newsock.getpeername())
            print(self.clients.keys())
            newsock.close()
            return
        for sock in self.clients.values():
            sock.send(data)

    def stop(self):
        self.socket.close()


if __name__ == '__main__':
    cc = ChatServer()
    cc.start()
    while True:
        events = cc.selector.select()
        print('events', events)
        for key, mask in events:
            key.data(key.fileobj, mask)